In [51]:
'''TODO List
Logic: 
1. Write output to xlsx instead of CSV 
2. Display the path of the final output file in a messagebox or a label? 
3. Error handling
    - Incorrect file selection (based on column names) - add elif to exception
    - file containing special characters that algorithm cannot process
4. Remove Index column before saving
'''

'TODO List\nLogic: \n1. Write output to xlsx instead of CSV \n2. Display the path of the final output file in a messagebox or a label? \n3. Error handling\n    - Incorrect file selection (based on column names) - add elif to exception\n    - file containing special characters that algorithm cannot process\n4. Remove Index column before saving\n'

In [52]:
import sys
print(sys.version)
print(sys.executable)
print (sys)

2.7.14 |Anaconda, Inc.| (default, Nov  8 2017, 13:40:45) [MSC v.1500 64 bit (AMD64)]
C:\ProgramData\Anaconda2\python.exe
<module 'sys' (built-in)>


In [53]:
#MENU 
import pandas as pd
import numpy as np
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import en

#Tkinter libraries
from Tkinter import *
from Tkinter import Menu
import tkFileDialog
import Tkconstants
import tkMessageBox

#Training libraries
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

#converting String to Pandas df
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
import datetime

In [54]:
#Training algorigthm
train_text = state_union.raw("2005-GWBush.txt")
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

In [55]:
#multiple 'should'
def transform_stmt_mul_shd(stmt):
    temp_pst_collate = ""
    splt_sentence = stmt.split("and")
    for stmt in splt_sentence:
        temp_pst_collate = temp_pst_collate + 'and' + transform_stmt_single_shd(stmt)
        #print(temp_pst_collate)
    return temp_pst_collate[3:]          

In [56]:
#Single 'should'
def transform_stmt_single_shd(stmt): 
    words = nltk.word_tokenize(stmt)
    veb_lst = [word for (word, pos) in nltk.pos_tag(words) if is_verb(pos)]
    
    if (stmt.find("should not") > -1) and ('be' in veb_lst):
        trnf_pst_stmt = stmt.replace("should not be", "is not")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("details") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("properties") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("fields") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("sections") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
         
    elif (stmt.find("should") > -1) and ('be' in veb_lst):
        trnf_pst_stmt = stmt.replace("should be", "is")
        return trnf_pst_stmt
        
    elif (stmt.find("should") > -1) and ('be' not in veb_lst):
        trnf_pst_stmt = stmt.replace("should "+veb_lst[0], en.verb.present(veb_lst[0].lower()))
        return trnf_pst_stmt
    
    else:
        return stmt

In [57]:
# creating a function that will retrive only base verb forms
is_verb = lambda pos: pos == 'VB' 

In [58]:
#process Expected result - convert to past tense
def processExpRes(df):
    #verb_lst = []
    all_past_stmt = []
    temp_pst = ""

    for test_case in list(df["Expect_res"]):
        stmt_lst = custom_sent_tokenizer.tokenize(test_case)
        try:
            for stmt in stmt_lst:
                words = nltk.word_tokenize(stmt)
                if words.count("should") > 1:
                    temp_pst = temp_pst + '\n' + transform_stmt_mul_shd(stmt)
                else:
                    temp_pst = temp_pst + '\n' + transform_stmt_single_shd(stmt)
                
            all_past_stmt.append(temp_pst[1:])
            #print (temp_pst)
            temp_pst = ""

        except Exception as e:
            all_past_stmt.append(test_case)
            print(str(e))
    se = pd.Series(all_past_stmt)
    df['Converted_Expected'] = se.values

In [59]:
#process actual result - trim User Info
def processActRes(df):
    trm_act_res = []
    for test_case in list(df["actualresult"]):
        try:
            #print (test_case)
            trm_act_res.append(test_case[:test_case.rfind("[")])
        except Exception as e:
            trm_act_res.append(test_case)
            print(str(e))   
    ac = pd.Series(trm_act_res)
    df['Converted_Actual'] = ac.values

In [60]:
def generatecompResult(df):
    cmpexpactlst = []
    resulttxt = ''
    for index, row in df.iterrows():
        #print(row["Converted_Expected"] +"----> \n"+  row["Converted_Actual"])

        x = row["Converted_Expected"].replace(' ', '')
        x1 = x.replace('\n', '') #replacing new lines
        x2 = x1.replace('\r', '')#replacing returns

        y = row["Converted_Actual"].replace(' ', '')
        y1 = y.replace('\n', '') #replacing new lines
        y2 = y1.replace('\r', '') #replacing returnss

        if x2 == y2:
            resulttxt = 'Approved'
        elif x2 in y2:
            #y2.contains(x2):
            resulttxt = 'Approved'
        else:
            resulttxt = 'Verify'
        cmpexpactlst.append(resulttxt)
    res = pd.Series(cmpexpactlst)
    df["Compare_result"] = res.values

In [66]:
#main fn
root = Tk()
root.title("test")

root.geometry('500x500')

def hello():
      print "hello!"

def close_window(): 
    root.destroy()

def generatefile():
    try:
        file = tkFileDialog.askopenfile(parent=root,mode='rb',title='Choose a file')
        if file != None:
            data = file.read()

            #convert file data from String to df
            TESTDATA = StringIO(data)
            df = pd.read_csv(TESTDATA, sep=",")

            #calling the process convert expected result function here
            processExpRes(df)
            print (df.shape)

            #calling the process convert act result fucntion here
            processActRes(df)
            print (df.shape)

            #calling generate comparison function here
            generatecompResult(df)
            print (df.shape)

            time = datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")
            print (time)

            df.to_csv('C:\\Users\\A6LBGZZ\\Desktop\\./FinalOutput_{}.csv'.format(time), encoding='utf-8')
            #df.to_excel('./FinalOutput_{}.xlsx'.format(time), encoding='utf-8')
            tkMessageBox.showinfo("Title", "HANU")

            file.close()
            print "I got %d bytes from this file." % len(data)
            
            from GeneratingSummary import hello

    except Exception as e:
        print(e)
        if e =="'ascii' codec can't decode byte 0xe2 in position 163: ordinal not in range(128)":
            tkMessageBox.showinfo("Error", "Cannot write to xls")

menu = Menu(root)
new_item = Menu(menu)

filemenu = Menu(menu, tearoff=0)
filemenu.add_command(label="Open", command=hello)
filemenu.add_command(label="Save", command=hello)
filemenu.add_separator()
filemenu.add_command(label="Exit", command=root.destroy)
menu.add_cascade(label="File", menu=filemenu)

brwsLbl1=Label(root)
brwsLbl1.grid(row=3, column=2)

#brwsBtn=Button(root, text="Browse", command=generatefile)
#brwsBtn.grid(row=2, column=1)

brwsLbl=Label(root)
brwsLbl.grid(row=3, column=2)

prcsBtn=Button(root, text="BROWSE & PROCESS", command=generatefile)
prcsBtn.grid(row=5, column=5)

exitBtn = Button(root, text = "EXIT", command = close_window)
exitBtn.grid(row=8, column=8)

#table = tktable.Table(root, rows=10, cols=4)
#table.pack(side="top", fill="both", expand=True)

root.config(menu=menu)
root.mainloop()

''
''
''
''
''
''
(1382, 5)
(1382, 6)
(1382, 7)
2018_04_27_225018
I got 345352 bytes from this file.
No module named GeneratingSummary
